In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Dataset/IDS 2018 Intrusion CSVs (CSE-CIC-IDS2018)/Expremente-Dataset/Dataset to Expreminte1.csv')

In [7]:
# بررسی و جایگزینی مقادیر بی‌نهایت با میانگین هر ستون
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data = data.dropna()

# برچسب گذاری Label به عددی
le = LabelEncoder()
data['Label'] = le.fit_transform(data['Label'])

# تقسیم ویژگی‌ها و برچسب‌ها
X = data.drop(['Label'], axis=1)
y = data['Label']

# نرمال سازی داده‌ها
scaler = StandardScaler()
X = scaler.fit_transform(X)

<ipython-input-7-20db8783a247>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Label'] = le.fit_transform(data['Label'])


In [8]:
# تشخیص داده‌های پرت با استفاده از Local Outlier Factor
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
outliers = lof.fit_predict(X)
X, y = X[outliers != -1], y[outliers != -1]  # حذف داده‌های پرت

In [23]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor

# پاک‌سازی داده‌ها: حذف مقادیر Null و تنظیم ایندکس‌ها
data_clean = data.dropna().reset_index(drop=True)

# برچسب‌گذاری برچسب‌های کلاس با استفاده از LabelEncoder
labelencoder = LabelEncoder()
data_clean['Label'] = labelencoder.fit_transform(data_clean['Label'])

# حذف مقادیر inf
data_clean = data_clean.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop=True)

# تبدیل DataFrame به numpy array و جدا کردن ویژگی‌ها و برچسب‌ها
X = data_clean.iloc[:, 0:78].values.astype("float32")
Y = data_clean.iloc[:, 79:].values

# برچسب‌ها را به One-Hot تبدیل می‌کنیم
enc = OneHotEncoder()
Y = enc.fit_transform(Y).toarray()

# نرمال‌سازی داده‌ها
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# تشخیص داده‌های پرت با استفاده از Local Outlier Factor (LOF)
lof = LocalOutlierFactor(contamination=0.05)
outliers = lof.fit_predict(X_scaled) == 1  # داده‌های بدون پرت

# حذف داده‌های پرت
X_cleaned = X_scaled[outliers]
Y_cleaned = Y[outliers]

# تقسیم داده‌ها به مجموعه‌های آموزش و تست
X_train, X_test, Y_train, Y_test = train_test_split(
    X_cleaned, Y_cleaned, test_size=0.2, random_state=2, shuffle=True
)

# ویژگی‌های ورودی و تعداد کلاس‌ها
_features = X.shape[1]
n_classes = Y.shape[1]

# مشاهده خلاصه‌ای از داده‌ها
print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of testing samples: {X_test.shape[0]}")
print(f"Number of features: {_features}")
print(f"Number of classes: {n_classes}")

Number of training samples: 85400
Number of testing samples: 21350
Number of features: 78
Number of classes: 15


In [25]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

# 2. الگوریتم XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=2)
xgb_model.fit(X_train, Y_train)

# پیش‌بینی و ارزیابی XGBoost
y_pred_xgb = xgb_model.predict(X_test)
print("XGBoost Results:")
print(classification_report(Y_test.argmax(axis=1), y_pred_xgb.argmax(axis=1)))
print(f"Accuracy (XGBoost): {accuracy_score(Y_test.argmax(axis=1), y_pred_xgb.argmax(axis=1)):.2f}")
print(f"MCC (XGBoost): {matthews_corrcoef(Y_test.argmax(axis=1), y_pred_xgb.argmax(axis=1)):.2f}")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:43:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Results:
              precision    recall  f1-score   support

           0       0.93      0.90      0.91      1329
           1       1.00      1.00      1.00      1483
           2       1.00      0.98      0.99      1221
           3       1.00      1.00      1.00      1511
           4       1.00      1.00      1.00      1453
           5       1.00      1.00      1.00      1447
           6       1.00      1.00      1.00      1474
           7       1.00      1.00      1.00      1508
           8       1.00      1.00      1.00      1483
           9       0.80      0.50      0.62      1455
          10       1.00      1.00      1.00      1287
          11       0.65      0.88      0.74      1514
          12       0.90      0.93      0.92      1289
          13       0.98      1.00      0.99      1433
          14       1.00      1.00      1.00      1463

    accuracy                           0.95     21350
   macro avg       0.95      0.95      0.94     21350
weighted 

In [30]:
# 3. الگوریتم SVM
svm_model = SVC(kernel='rbf', random_state=2, probability=True)  # kernel='linear' یا 'poly' نیز قابل استفاده است
svm_model.fit(X_train, Y_train.argmax(axis=1))  # برای SVM، برچسب‌ها به‌صورت تک‌ستونه نیاز است

# پیش‌بینی و ارزیابی SVM
y_pred_svm = svm_model.predict(X_test)
print("SVM Results:")
print(classification_report(Y_test.argmax(axis=1), y_pred_svm))
print(f"Accuracy (SVM): {accuracy_score(Y_test.argmax(axis=1), y_pred_svm):.2f}")
print(f"MCC (SVM): {matthews_corrcoef(Y_test.argmax(axis=1), y_pred_svm):.2f}")

SVM Results:
              precision    recall  f1-score   support

           0       0.82      0.70      0.75      1329
           1       1.00      1.00      1.00      1483
           2       0.84      0.70      0.76      1221
           3       0.65      0.95      0.77      1511
           4       0.99      1.00      1.00      1453
           5       1.00      0.98      0.99      1447
           6       1.00      1.00      1.00      1474
           7       1.00      1.00      1.00      1508
           8       1.00      1.00      1.00      1483
           9       0.59      0.99      0.74      1455
          10       1.00      1.00      1.00      1287
          11       0.98      0.34      0.50      1514
          12       0.79      0.77      0.78      1289
          13       0.86      0.71      0.78      1433
          14       1.00      1.00      1.00      1463

    accuracy                           0.88     21350
   macro avg       0.90      0.88      0.87     21350
weighted avg 

In [34]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
# 3. الگوریتم KNN
knn_model = KNeighborsClassifier(n_neighbors=5)  # تعداد همسایگان را می‌توان بسته به نیاز تنظیم کرد
knn_model.fit(X_train, Y_train.argmax(axis=1))  # برچسب‌ها برای KNN باید به‌صورت تک‌ستونه باشند
# پیش‌بینی و ارزیابی KNN
y_pred_knn = knn_model.predict(X_test)
print("KNN Results:")
print(classification_report(Y_test.argmax(axis=1), y_pred_knn))
print(f"Accuracy (KNN): {accuracy_score(Y_test.argmax(axis=1), y_pred_knn):.2f}")
print(f"MCC (KNN): {matthews_corrcoef(Y_test.argmax(axis=1), y_pred_knn):.2f}")

KNN Results:
              precision    recall  f1-score   support

           0       0.92      0.86      0.89      1329
           1       1.00      1.00      1.00      1483
           2       0.99      0.98      0.99      1221
           3       1.00      1.00      1.00      1511
           4       1.00      1.00      1.00      1453
           5       1.00      1.00      1.00      1447
           6       1.00      1.00      1.00      1474
           7       1.00      1.00      1.00      1508
           8       1.00      1.00      1.00      1483
           9       0.79      0.51      0.62      1455
          10       1.00      1.00      1.00      1287
          11       0.65      0.87      0.74      1514
          12       0.87      0.92      0.89      1289
          13       0.98      1.00      0.99      1433
          14       1.00      1.00      1.00      1463

    accuracy                           0.94     21350
   macro avg       0.95      0.94      0.94     21350
weighted avg 